In [32]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/kuet-whispers/whispers_eng.csv
/kaggle/input/kuet-whispers/whispers.csv
/kaggle/input/kuet-whispers/whispers_Bangla.csv


In [33]:
df = pd.read_csv('/kaggle/input/kuet-whispers/whispers.csv')
print(df)

      post no                                               text  \
0      2443.0  #2443\nI wanna write “I miss you” on a rock an...   
1      2442.0  #2442\nsab ko mil jaegi manzil, ye zaruri to n...   
2      2441.0  #2441\nThey say, "Our expectation is someone e...   
3      2440.0  #2440\nআমি কুয়েটে আসার আগে ভাবতাম মেয়েরা সব থে...   
4      2439.0  #2439\nAny female FPL manager here in KUET?\n\...   
...       ...                                                ...   
2480      5.0  #005\nThe transition from smarty-pants to poop...   
2481      4.0  #004\nMy chances of passing this semester is t...   
2482      3.0  #003\nThe only welfare you are gonna get in yo...   
2483      2.0  #002\nI had the liberty to choose between "Ana...   
2484      1.0  #001\nThe antonym for Keut has to be KUET\n-an...   

                                           cleaned_text  likes  comments  \
0     I wanna write “I miss you” on a rock and throw...    182       5.0   
1     sab ko mil jaegi manzil, 

In [34]:
df

,post no,text,cleaned_text,likes,comments,sender,time,url
0,2443.0,#2443\nI wanna write “I miss you” on a rock an...,I wanna write “I miss you” on a rock and throw...,182,5.0,an angry bird,2025-07-05 16:02:30,https://www.facebook.com/61554694872000/posts/...
1,2442.0,"#2442\nsab ko mil jaegi manzil, ye zaruri to n...","sab ko mil jaegi manzil, ye zaruri to nahi ~No...",20,1.0,Anonymous,2025-07-05 15:57:19,https://www.facebook.com/61554694872000/posts/...
2,2441.0,"#2441\nThey say, ""Our expectation is someone e...","They say, ""Our expectation is someone else's r...",62,0.0,"Your Well-Wisher, Forever",2025-07-05 15:56:35,https://www.facebook.com/61554694872000/posts/...
3,2440.0,#2440\nআমি কুয়েটে আসার আগে ভাবতাম মেয়েরা সব থে...,আমি কুয়েটে আসার আগে ভাবতাম মেয়েরা সব থেকে বেশি...,874,49.0,anonymous,2025-07-05 15:56:09,https://www.facebook.com/61554694872000/posts/...
4,2439.0,#2439\nAny female FPL manager here in KUET?\n\...,Any female FPL manager here in KUET?,3,0.0,anonymous,2025-07-05 15:55:42,https://www.facebook.com/61554694872000/posts/...
...,...,...,...,...,...,...,...,...
2480,5.0,#005\nThe transition from smarty-pants to poop...,The transition from smarty-pants to poopy-pant...,4,0.0,NaN,2023-12-24 15:53:34,https://www.facebook.com/61554694872000/posts/...
2481,4.0,#004\nMy chances of passing this semester is t...,My chances of passing this semester is the fem...,20,0.0,NaN,2023-12-24 15:01:22,https://www.facebook.com/61554694872000/posts/...
2482,3.0,#003\nThe only welfare you are gonna get in yo...,The only welfare you are gonna get in your KUE...,19,0.0,NaN,2023-12-24 13:27:20,https://www.facebook.com/61554694872000/posts/...
2483,2.0,"#002\nI had the liberty to choose between ""Ana...","I had the liberty to choose between ""Anatomy m...",21,1.0,NaN,2023-12-24 11:59:23,https://www.facebook.com/61554694872000/posts/...


In [35]:
import pandas as pd
import re

# 1. Load your data
df = pd.read_csv('/kaggle/input/kuet-whispers/whispers.csv', encoding='utf-8')

# 2. Compile regexes for Bangla and English
bn_pattern = re.compile(r'[\u0980-\u09FF]')   # any Bangla character
en_pattern = re.compile(r'[A-Za-z]')           # any Latin letter

def classify_language(text):
    """
    Classify a passage as Bangla or English by:
     1) Counting how many words contain at least one Bangla vs. English character.
     2) If tied, falling back to comparing total character counts.
    """
    s = str(text)
    words = s.split()
    
    # Count words containing Bangla or English characters
    bn_word_count = sum(bool(bn_pattern.search(w)) for w in words)
    en_word_count = sum(bool(en_pattern.search(w)) for w in words)

    if bn_word_count > en_word_count:
        return 'Bangla'
    if en_word_count > bn_word_count:
        return 'English'
    
    # Tie‑breaker: compare total character counts
    bn_char_count = len(bn_pattern.findall(s))
    en_char_count = len(en_pattern.findall(s))
    return 'Bangla' if bn_char_count > en_char_count else 'English'

# 3. Apply classification
df['language'] = df['cleaned_text'].apply(classify_language)

# 4. Split into two DataFrames
df_bangla  = df[df['language'] == 'Bangla']
df_english = df[df['language'] == 'English']

# 5. Save to CSV
df_bangla.to_csv('whispers_Bangla.csv',  index=False, encoding='utf-8')
df_english.to_csv('whispers_eng.csv', index=False, encoding='utf-8')

print(f"Bangla passages:  {len(df_bangla)} saved ➔ whispers_Bangla.csv")
print(f"English passages: {len(df_english)} saved ➔ whispers_eng.csv")


Bangla passages:  747 saved ➔ whispers_Bangla.csv
English passages: 1738 saved ➔ whispers_eng.csv


In [36]:
df_bn = pd.read_csv('whispers_Bangla.csv')
df_en = pd.read_csv('whispers_eng.csv')


In [37]:
print(df_bn)

     post no                                               text  \
0     2440.0  #2440\nআমি কুয়েটে আসার আগে ভাবতাম মেয়েরা সব থে...   
1     2434.0  #2434\nআমি জানিনা এখানে কেনো লিখছি,কোথাও বলার ...   
2     2432.0  #2432\nএকটা চিঠি, যা লেখা হয়নি\n\nরাতটা ছিল ন...   
3     2431.0  #2431\nশ্রদ্ধাসহ স্মরণ করি সেই আমার পুং-শ্রেণি...   
4     2424.0  #2424\nOverthinking যে মানুষকে পাগল বানিয়ে ছা...   
..       ...                                                ...   
742     28.0  #028\n১-১ তো শেষই বলা যায়। না হলো প্রেম; না পড়...   
743     27.0  #027\nYes, it's a threat.\nচুইঝাল ট্রিট না দিল...   
744     25.0  #025\nঘুমিয়ে গেছি আবারো, কে করবে আমার অ্যাসাই...   
745     21.0  #021\nকুয়েট দিয়েছে কেবল এক রাশ নি:সঙ্গতা।\n-an...   
746     19.0  #019\nকুয়েটের ক্লাস পরীক্ষা বন্ধ তাই reels দে...   

                                          cleaned_text  likes  comments  \
0    আমি কুয়েটে আসার আগে ভাবতাম মেয়েরা সব থেকে বেশি...    874      49.0   
1    আমি জানিনা এখানে কেনো লিখছি,কোথাও বলার ন

In [38]:
fifth_row = df_bangla.iloc[4]
print(fifth_row)

# 2. If you only care about the text column:
fifth_text = df_bangla['cleaned_text'].iloc[4]
print(fifth_text)

post no                                                    2424.0
text            #2424\nOverthinking যে মানুষকে পাগল বানিয়ে ছা...
cleaned_text    Overthinking যে মানুষকে পাগল বানিয়ে ছাড়ে এর ...
likes                                                          63
comments                                                      0.0
sender                                                  anonymous
time                                          2025-07-05 07:45:28
url             https://www.facebook.com/61554694872000/posts/...
language                                                   Bangla
Name: 20, dtype: object
Overthinking যে মানুষকে পাগল বানিয়ে ছাড়ে এর অন্যতম উদাহরণ আমি। একটা উদাহরণ দেই। আমার এক ফ্লাটমেট বাসা ছেড়ে দিবে। হঠাৎ একদিন মেয়েদের গ্রুপে খেয়াল করলাম একজন রুমমেট খুঁজতেছে। পোস্টদাতা আমার ফ্লাটমেটেরই ডেপ্ট এর। আমারও মনে পড়ল কিছুদিন আগে আমার রুমমেট ও গ্রুপে একই বাসার পোস্ট দিয়েছিলো। আমার overthinking mind খুব তাড়াতাড়ি হিসেব নিকেশ শুরু করে দিল। হয়তো আমার ফ্লাটমেটই রুমমেট 

In [39]:
fifth_row = df_english.iloc[4]
print(fifth_row)

# 2. If you only care about the text column:
fifth_text = df_english['cleaned_text'].iloc[4]
print(fifth_text)

post no                                                    2438.0
text            #2438\nDrop some thriller movie/series suggest...
cleaned_text         Drop some thriller movie/series suggestions 
likes                                                          16
comments                                                     65.0
sender                                                  anonymous
time                                          2025-07-05 15:55:14
url             https://www.facebook.com/61554694872000/posts/...
language                                                  English
Name: 5, dtype: object
Drop some thriller movie/series suggestions 
